In [ ]:
import sys
sys.path.append('/home/cucchi/phd/devel/l96')

In [ ]:
import lab.simulation as sim
import xarray as xr
import math
import numpy as np

In [ ]:
DATA_PATH = '/home/cucchi/phd/data/'

Run simulation with constant forcing. In this case, I keep all dimensions since I will use values as initial conditions for simulations with other forcings.

In [ ]:
out_path_const = '{}sim_const_F8.nc'.format(DATA_PATH)

point_const = sim.SystemState(coords=[8]*32)
point_const.perturbate()
constant_force = sim.ConstantForcing()
simulator_const = sim.Simulator(system_state=point_const, forcing=constant_force)

runner = sim.SimulationRunner(
    simulator=simulator_const, 
)
runner.run(
    out_file=out_path_const,
    integration_time = 10000,
    chunk_length = 10000,
    write_all = True
)

Load simulation

In [ ]:
data_const = xr.open_dataarray(out_path_const)

Select initial conditions for other simulations at times multiples of 100

In [ ]:
single_experiment_time_length=100
inits = data_const.where((data_const.time%single_experiment_time_length)==0, drop=True)

Use inits to start new simulations with delta forcing

In [ ]:
for i in np.arange(0, len(inits.time)):
    out_path_delta = '{}delta_{}.nc'.format(DATA_PATH, str(i).zfill(3))
    point_delta = sim.SystemState(coords=inits.isel(time=i).values, time=float(inits.time[i].values))
    delta_force = sim.DeltaForcing(activation_time=float(inits.time[i].values))
    simulator_delta = sim.Simulator(system_state=point_delta, forcing=delta_force)
    runner = sim.SimulationRunner(
        simulator=simulator_delta, 
    )
    runner.run(
        out_file=out_path_delta,
        integration_time = single_experiment_time_length,
        chunk_length = 10000,
        write_all = False
    )

Load all and concat

In [ ]:
dataarrays = []
for i in np.arange(0, len(inits.time)):
    in_path_delta = '{}delta_{}.nc'.format(DATA_PATH, str(i).zfill(2))
    dataarrays.append(xr.open_dataarray(in_path_delta))
data_delta = xr.concat(dataarrays, dim='time')

Compute energies and keep only node 0

In [ ]:
energy_const = 0.5*(data_const.sel(node=0)**2)
energy_delta = 0.5*(data_delta**2)

Compute average response among each simulation

In [ ]:
response = energy_delta - energy_const
original_times = response.time
time_indices = (np.arange(0, len(response.time))).astype(int) % int(single_experiment_time_length/0.01)
response = response.assign_coords(time=time_indices)
response_mean = response.groupby('time').apply(np.mean)
response_mean.plot()

Do the same as before, but with step forcing

In [ ]:
for i in np.arange(0, len(inits.time)):
    out_path_step = '{}step_{}.nc'.format(DATA_PATH, str(i).zfill(2))
    point_step = sim.SystemState(coords=inits.isel(time=i).values, time=float(inits.time[i].values))
    step_force = sim.StepForcing(activation_time=float(inits.time[i].values))
    simulator_step = sim.Simulator(system_state=point_step, forcing=step_force)
    runner = sim.SimulationRunner(
        simulator=simulator_step, 
    )
    runner.run(
        out_file=out_path_step,
        integration_time = single_experiment_time_length,
        chunk_length = 10000,
        write_all = False
    )

dataarrays = []
for i in np.arange(0, len(inits.time)):
    in_path_step = '{}step_{}.nc'.format(DATA_PATH, str(i).zfill(2))
    dataarrays.append(xr.open_dataarray(in_path_step))
data_step = xr.concat(dataarrays, dim='time')

energy_step = 0.5*(data_step**2)

response = energy_step - energy_const
original_times = response.time
time_indices = (np.arange(0, len(response.time))).astype(int) % int(single_experiment_time_length/0.01)
response = response.assign_coords(time=time_indices)
response_mean_step = response.groupby('time').apply(np.mean)
response_mean_step.plot()